In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import tqdm
import string
import re

import warnings
warnings.filterwarnings("error")
warnings.filterwarnings("ignore")

In [2]:
import spacy
import textacy

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
# Read the data and drop the null values

review_df = pd.read_csv("/home/heptagon/Desktop/nps_analysis/reviews_data.csv")
review_df = review_df.dropna().reset_index(drop=True)
df_text = review_df['CleanedText']

In [4]:
review_df

,Unnamed: 0,comments,CleanedText
0,0,The material is really in accordance with the ...,The material is really in accordance with the ...
1,1,Overall bag is good but.. now these days .. th...,Overall bag is good but. now these days . this...
2,2,Good material with zipper quality,Good material with zipper quality
3,3,👍,👍
4,4,I bought it for 700 something which is great d...,I bought it for 700 something which is great d...
...,...,...,...
4915,4995,Very Nice material and very good space to keep...,Very Nice material and very good space to keep...
4916,4996,Good,Good
4917,4997,Awsome,Awsome
4918,4998,Awesome product I am happy super quality,Awesome product I am happy super quality


In [36]:
def get_clean_data(df):
    clean_text = []
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-Z]', ' ', str(df[i]))
        review = " ".join(re.split("\s+", review, flags=re.UNICODE))

        if review != ' ':
            clean_text.append(review)  

    return clean_text

get_clean_data(df_text)

['The material is really in accordance with the brand name Storage is good and the back padding is good just miss the laptop sleeves ',
 'Overall bag is good but now these days this bag is not good In rainy days you don t get a rain cover for this bag ',
 'Good material with zipper quality',
 'I bought it for something which is great deal And it is great for travelling purpose rather than daily useage',
 'Reasonable n useful',
 'Good product in its price',
 'Super',
 'Good product for gift',
 'Good as we expected weight less ',
 'Best of best ',
 'Good product',
 'It s a good product in a true price',
 'Product is goodCons No rain protection cover',
 'Good',
 'MRP but rs price buying Good quality material bagOverall good ',
 'The bag is good for capacity and it is very good for travelling to take even to school it s very good but tear in one month or two months request to not by product you can choose another product please don t buy this product event is going to buy we will take it c

In [33]:
# for sent in clean_text:
#     if (len(sent) == 1):
#         print(sent) 


A
A


In [46]:
df = pd.DataFrame(clean_text, columns=['CleanedText'])
df

,CleanedText
0,the material is really in accordance with the ...
1,overall bag is good but now these days this ba...
2,good material with zipper quality
3,i bought it for something which is great deal ...
4,reasonable n useful
...,...
4880,very nice material and very good space to keep...
4881,good
4882,awsome
4883,awesome product i am happy super quality


In [16]:
# lst = []
# for i in range(len(clean_text)):
#     lst.append([clean_text[i]])

# lamma = []
# for data in lst:
#     lemmatizer = WordNetLemmatizer()        # Apply Lemmatization
#     review = [lemmatizer.lemmatize(word) for word in data]
#     review = ' '.join(review)

In [47]:
# # Remove Special Characters, Convert into the lower case and Stop Words & Apply Lemmatization

def clean_text_and_lemmatize(df):

    stop_words = stopwords.words('english')    # list of Stop Words Excluding (not, no)
    stop_words.remove("not")
    stop_words.remove("no")

    lemmatizer = WordNetLemmatizer()        # Apply Lemmatization
    corpus_list = []

    for i in range(0, len(clean_text)):
        review = str(clean_text[i]).lower()
        review = review.split()        

        review = [lemmatizer.lemmatize(word) for word in review if not word in stop_words]
        review = ' '.join(review)
        corpus_list.append(review)

    return corpus_list

key_corpys_list = clean_text_and_lemmatize(clean_text)


the material is really in accordance with the brand name storage is good and the back padding is good just miss the laptop sleeves 
overall bag is good but now these days this bag is not good in rainy days you don t get a rain cover for this bag 
good material with zipper quality
i bought it for something which is great deal and it is great for travelling purpose rather than daily useage
reasonable n useful
good product in its price
super
good product for gift
good as we expected weight less 
best of best 
good product
it s a good product in a true price
product is goodcons no rain protection cover
good
mrp but rs price buying good quality material bagoverall good 
the bag is good for capacity and it is very good for travelling to take even to school it s very good but tear in one month or two months request to not by product you can choose another product please don t buy this product event is going to buy we will take it careful or like for travelling don t take for school why i said

good
a quality product by amarican tourister leather are also good quality 
the white paint on top has started going from the second week itself 
very nice bag size fit
product is good but delivered post followup
overall ok 
good
has a lot of compartments value for money 
good
it s very light weight easy to carry and it s very good in colour 
good product light weight big i wish it had a slot to put laptop too 
design and size of bag is good but the quality is low compared to what i have used from this brand before zippers are weak 
good product
good
nice
light weight but quality of material is not up to mark
very good quality it s being more than a year and still it is going strong no doubts u can go for it
good product under 
below average bag not woth the value don t waste your money on this
go for it
ok for rs
very good product at reasonablw price
i faced no problem in but i am still have some doubt
ok product
anterior holder chain gets stuck in inner pocket fold while closing it v

In [35]:
key_corpys_list

['material really accordance brand name storage good back padding good miss laptop sleeve',
 'overall bag good day bag not good rainy day get rain cover bag',
 'good material zipper quality',
 'bought something great deal great travelling purpose rather daily useage',
 'reasonable n useful',
 'good product price',
 'super',
 'good product gift',
 'good expected weight le',
 'best best',
 'good product',
 'good product true price',
 'product goodcons no rain protection cover',
 'good',
 'mrp r price buying good quality material bagoverall good',
 'bag good capacity good travelling take even school good tear one month two month request not product choose another product please buy product event going buy take careful like travelling take school said not take district school take school get automatically tear four month aur month product please travelling buy take school',
 'travel',
 'nice school bag',
 'light weight comfortable',
 'show raincover actually water resistant material no rai

### Part of speech with diff patterns

In [33]:
# Get Keywords & Apply Part of Speech  

def get_keywords_phreses(key_phreses):

    nlp = spacy.load("en_core_web_sm")
    key_list_phreses = []

    for data in range(0, len(key_phreses)):
        doc = nlp(key_phreses[data])

        patterns_phrases = [[{"POS":"ADJ"}, {"POS":"NOUN"}] or [{"POS":"ADJ"}] or [{"POS":"NOUN"}]]

        combined_phrases_pattern = textacy.extract.matches.token_matches(doc, patterns=patterns_phrases)

        key_phrase_combined = [str(combined_phrase) for combined_phrase in combined_phrases_pattern]

        combined_str = ",".join(key_phrase_combined)
        key_list_phreses.append(combined_str)

    return key_list_phreses

key_phreses_list = get_keywords_phreses(key_corpys_list)

# {"POS":"NOUN"}, {"POS":"PRON"}, {"POS":"VERB"}, {"POS":"ADV"}

In [34]:
print(key_phreses_list)

['good miss', 'overall bag,good day,rainy day', 'good material', 'great deal,daily useage', '', 'good product', '', 'good product', '', '', 'good product', 'good product,true price', '', '', 'good quality', 'good capacity,good tear', '', 'nice school', 'light weight', 'resistant material', 'outer fabric,resistant water', 'amazing product', 'small packet', 'good product,net way', '', 'enough brand,light weight', 'good product,good storage', 'better bag', 'good buy', '', 'best loglasting', 'lightweight zipper,enough space,many pocket', 'disappointed fact', '', '', 'good product', '', 'good performance', 'blue bag,perfect need,easy pack,heavy textbook,great product', '', '', 'durable bag', '', 'good one', 'lite wait,american trurister,white color', 'overall bag,perfect choice,expensive quality,good nd', 'good quality', '', '', '', '', 'good price', 'nice water', '', 'good quality,available laptop', 'good size', '', 'good condition', '', 'good bag', 'old son,happy stay,academic year', '', 

##### Apply Hugging face Zero-Shot-Classification 

In [36]:
from transformers import pipeline

In [37]:
# Apply Zero Shot Classifier using hugging face

def type_classifier_model(corpus_list, type_list):

    classifier = pipeline("zero-shot-classification", model='typeform/distilbert-base-uncased-mnli')
    res = classifier(sequences=corpus_list[:10], candidate_labels=type_list, multi_class=True) 

    type_list = []
    for data in res:
        review_type = dict(zip(data['labels'], data['scores']))
        type_list.append(max(review_type, key = review_type.get))

    return type_list

In [38]:
type_list = ['appreciation', 'information', 'complaint']

type_classifier_list = type_classifier_model(key_corpys_list, type_list)
type_classifier_list

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


['information',
 'appreciation',
 'appreciation',
 'appreciation',
 'appreciation',
 'information',
 'appreciation',
 'appreciation',
 'information',
 'appreciation']

##### Sentiment Classifier

In [32]:
# Apply Sentiment Classifier Model

def get_sentiment_classifier_model(key_corpys_list):
    sentiment_classifier = pipeline("sentiment-analysis")
    sentiment_res = sentiment_classifier(key_corpys_list)

    sentiment_label_list = []
    for i in sentiment_res:
        sentiment_label_list.append(i['label'])
    
    return sentiment_label_list

In [33]:
sentiment_classifier_list = get_sentiment_classifier_model(key_corpys_list)
sentiment_classifier_list

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


['POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',

In [34]:
key_phrase_df = review_df.copy()

key_phrase_df['topic'] = key_phreses_list
key_phrase_df['type'] = type_classifier_list
key_phrase_df['sentiment'] = sentiment_classifier_list

In [35]:
key_phrase_df.to_csv("/workspace/nps_analysis/key_phrases.csv", index=False)